In [1]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, count, rand, collect_list, explode, struct, count

In [2]:
from pyspark.sql.functions import pandas_udf

df = spark.range(0, 10 * 10 * 100).withColumn('id', (col('id') / 1000).cast('integer')).withColumn('v', rand())

df.cache()
df.count()

In [3]:
display(df.groupBy("id").count())

In [4]:
df.registerTempTable("df")

In [5]:
grouped_df = spark.sql("select id,collect_list(v) as ts from df group by id")
grouped_df.registerTempTable("grouped_df")

In [6]:
cross_df = spark.sql("select x.id as id1, x.ts as ts1, y.id as id2, y.ts as ts2 from (select * from grouped_df) x cross join (select * from grouped_df) y")
cross_df.registerTempTable("cross_df")

In [7]:
from dtaidistance import dtw
def dtw_dist1(s1,s2):
  return dtw.distance(s1, s2)
spark.udf.register("dtw_dist1",dtw_dist1,"double")

In [8]:
cross_df.cache()
cross_df.collect()
cross_df.count()

In [9]:
dtw_dist1_udf = udf(dtw_dist1,"double")

In [10]:
display(cross_df.withColumn("dist1",dtw_dist1_udf(col("ts1"),col("ts2"))).select("id1","id2","dist1"))

In [11]:
fastdtw_dist1_udf = udf(fastdtw_dist1,"double")

In [12]:
cross_df.withColumn("dist1",dtw_dist1_udf(col("ts1"),col("ts2"))).select("id1","id2","dist1")

In [13]:
cross_df.withColumn("dist1",fastdtw_dist1_udf(col("ts1"),col("ts2"))).select("id1","id2","dist1")

In [14]:
display(cross_df.withColumn("dist1",fastdtw_dist1_udf(col("ts1"),col("ts2"))).select("id1","id2","dist1"))

In [15]:
%sql

select id1,id2,dtw_dist1(ts1,ts2) from cross_df

In [16]:
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
def fastdtw_dist1(s1,s2):
  distance, path = fastdtw(s1, s2, dist=euclidean)
  return distance
spark.udf.register("fastdtw_dist1",fastdtw_dist1,"double")

In [17]:
spark.sql("select id1,id2,fastdtw_dist1(ts1,ts2) from cross_df")

In [18]:
%sql

select id1,id2,fastdtw_dist1(ts1,ts2) from cross_df

In [19]:
def my_custom_norm(x, y):
  x = np.array(x)
  y = np.array(y)
#   return (x * x) + (y * y)
  return norm(x-y)


dist, cost, acc, path = dtw(cross_df.take(1)[0][1], cross_df.take(2)[1][3], dist=my_custom_norm)
dist

In [20]:
a,b = cross_df.take(1)[0][1], cross_df.take(2)[1][3]

In [21]:
dist, cost, acc, path = dtw(a, b, dist=my_custom_norm)
dist

In [22]:
from dtaidistance import dtw
s1 = [0, 0, 1, 2, 1, 0, 1, 0, 0]
s2 = [0, 1, 2, 0, 0, 0, 0, 0, 0]
distance = dtw.distance(s1, s2)
print(distance)

In [23]:
from math import sqrt
def DTWDistance(s1, s2):
    DTW={}
    
    for i in range(len(s1)):
        DTW[(i, -1)] = float('inf')
    for i in range(len(s2)):
        DTW[(-1, i)] = float('inf')
    DTW[(-1, -1)] = 0

    for i in range(len(s1)):
        for j in range(len(s2)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
		
    return sqrt(DTW[len(s1)-1, len(s2)-1])

In [24]:
DTWDistance(a,b)

In [25]:
distance = dtw.distance(a, b)
distance

In [26]:

import pandas as pd
import numpy as np
import matplotlib.pylab as plt

x=np.linspace(0,50,100)
ts1=pd.Series(3.1*np.sin(x/1.5)+3.5)
ts2=pd.Series(2.2*np.sin(x/3.5+2.4)+3.2)
ts3=pd.Series(0.04*x+3.0)

In [27]:
distance1, path = fastdtw(ts1, ts2,dist=euclidean)
distance2, path = fastdtw(ts1, ts3,dist=euclidean)
print distance1, distance2

In [28]:
DTWDistance(ts1,ts2), DTWDistance(ts1,ts3)

In [29]:
21.549494824404572/17.929718468641138

In [30]:
206.59539301/ 144.758153526

In [31]:
lambda a, b: sum((a - b) ** 2)

In [32]:
distance, path = fastdtw(a, b,dist=my_custom_norm)
distance

In [33]:
distance, path = fastdtw(a, b, dist=euclidean)
distance

In [34]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

x = np.array([[1,1], [2,2], [3,3], [4,4], [5,5]])
y = np.array([[2,2], [3,3], [4,4]])
# distance, path = fastdtw(x, y, dist=euclidean)
distance, path = fastdtw(a, b, dist=euclidean)
print(distance)

In [35]:
dist

In [36]:
import numpy as np
from numpy.linalg import norm

In [37]:
len(cross_df.take(1)[0][1])